# Testing ways to automate date parsing 

We currently make machine-readable(ish) dates in the `YYYY-MM-DD` form for "Begin Date" and "End Date". When they are of a common form, these dates are relatively easily generated. However, like all the metadata we're provided, it's not in a consistent form. This notebook will explore tools like the Python package `datefinder` to see if it can help our common use cases 

## First up: `datefinder` 
The first package we'll try out is `datefinder`, so let's import it

In [65]:
import datefinder

We don't know anything about `datefinder` beyond its sparse [documentation](https://datefinder.readthedocs.io/en/latest/) about one function, `find_dates()`. Let's read it! 

In [34]:
help(datefinder.find_dates)

Help on function find_dates in module datefinder:

find_dates(text, source=False, index=False, strict=False, base_date=None)
    Extract datetime strings from text
    
    :param text:
        A string that contains one or more natural language or literal
        datetime strings
    :type text: str|unicode
    :param source:
        Return the original string segment
    :type source: boolean
    :param index:
        Return the indices where the datetime string was located in text
    :type index: boolean
    :param strict:
        Only return datetimes with complete date information. For example:
        `July 2016` of `Monday` will not return datetimes.
        `May 16, 2015` will return datetimes.
    :type strict: boolean
    :param base_date:
        Set a default base datetime when parsing incomplete dates
    :type base_date: datetime
    
    :return: Returns a generator that produces :mod:`datetime.datetime` objects,
        or a tuple with the source text and index, if req

We'll make a toy example with a text blob with our common use cases to see what happens

In [61]:
import datefinder
dates = ["2000/10/01", "February 12 2010", "Around March 2020", "3rd of March 2005"]
for d in dates:
    dates_found = datefinder.find_dates(d)
    for i in dates_found:
        print(i)

2000-10-01 00:00:00
2010-02-12 00:00:00
2020-03-11 00:00:00
2005-03-03 00:00:00


Hmmm. These results aren't great. It doesn't handle imbiguity very well, and when it seems to fail around "days," it seems to default to the 11th of the month.  

Since there's not really any other options or functions to use, it looks like `datefinder` is out. 

## Next: `dateparser` 
We also can try to use `dateparser`, which sounds similar but seems to have way more options from its [documentation](https://dateparser.readthedocs.io/en/latest). Lets try it.

In [13]:
from dateparser.search import search_dates
from dateparser import parse

dates = search_dates('Central design committee session Tuesday 10/22 6:30 pm, and last April 2019, Around March 2020, 3rd of March 2005')
for d in dates:
    print(d)

('Tuesday 10/22 6:30 pm, and', datetime.datetime(2020, 10, 22, 18, 30))
('April 2019', datetime.datetime(2019, 4, 22, 0, 0))
('March 2020', datetime.datetime(2020, 3, 22, 0, 0))
('3rd of March 2005', datetime.datetime(2005, 3, 3, 0, 0))


This parsing is kind of interesting, and the great thing about `dateparser` is it has settings we can specify. One in particular that seems useful is the ability to set defaults for incomplete or ambiguous dates. This would help us parse dates in the form of `YYYY-MM` or even in its plain text form, as above "April 2019." 

In [11]:
ambig_dates = search_dates('April 2019, 2017', settings={'PREFER_DAY_OF_MONTH': 'first'})
for d in ambig_dates:
    print(d)

('April 2019', datetime.datetime(2019, 4, 1, 0, 0))
('2017', datetime.datetime(2017, 4, 1, 0, 0))


As we can see in the output, instead of "April 22" as was parsed in our first example, since we specified `first` as a preferred day of the month, the day is now parsed as "1". This is exactly what we do in the frequent case where we are provided with just a month and a year.  

### Formatting
You may notice that the output has a lot of information, but is perhaps a bit awkwardly formatted. We should check the `type()` of data we're seeing, so we know how to process it into a better form.

In [18]:
ambig_dates = search_dates('April 2019, 2017', settings={'PREFER_DAY_OF_MONTH': 'first'})
for d in ambig_dates:
    print(d, type(d))

('April 2019', datetime.datetime(2019, 4, 1, 0, 0)) <class 'tuple'>
('2017', datetime.datetime(2017, 4, 1, 0, 0)) <class 'tuple'>


Ah, tuples. We know that we can "access" tuple parts the same way we splice lists. It looks like there's two elements (thus "**tu**ple!"), which would mean we probably need the second item, accessed via `tuple[1]` Let's try to splice these dates and see what comes back.

In [4]:
ambig_dates = search_dates('April 2019, 2017', settings={'PREFER_DAY_OF_MONTH': 'first'})
for d in ambig_dates:
    print("The original tuple form is:", d, "\n", "The spliced version is:", d[1])

The original tuple form is: ('April 2019', datetime.datetime(2019, 4, 1, 0, 0)) 
 The spliced version is: 2019-04-01 00:00:00
The original tuple form is: ('2017', datetime.datetime(2017, 4, 1, 0, 0)) 
 The spliced version is: 2017-04-01 00:00:00


In [7]:
ambig_dates = search_dates('April 2019, February 2017', settings={'PREFER_DAY_OF_MONTH': 'last'})
for d in ambig_dates:
    print(d[1])

2019-04-30 00:00:00
2017-02-28 00:00:00


This is potentially very useful. We can now parse month-specific dates, in one case favoring the first of the month, then the last, which will give us the proper Begin/End dates such as `2017-02-01` / `2017-02-28` given `February 2017`. 

## Handling conditionals
Now that we can deal with vague dates, we can start to compose a loop where we try to account for almost all date possibilities. Things we can't handle we'll just leave blank.  

Speaking of, how will we handle blank date cells? Traditional Python ways like checking length via `len() == 0` might not work due to data types. Perhaps `pandas` will provide us with a way!

In [2]:
import pandas as pd
import os

Let's see where we're at directory wise

In [4]:
os.getcwd()

'/home/zelgius/Github/metadata-work/documentation'

We'll change into our `/test` directory to use those nice test sheets

In [7]:
os.chdir("../test/")
os.listdir()

['catalhoyuk_glimpse_v2_delimiters.xlsx',
 'double_spaces.csv',
 'untrimmed_spaces.xlsx',
 'double_spaces.xlsx',
 '.ipynb_checkpoints',
 'untrimmed_spaces.csv']

In [8]:
dates_df = pd.read_excel('double_spaces.xlsx')

In [9]:
dates_df.head()

,Object Unique ID,Level,File name,File use,Type of Resource,Language,Title,Person:Creator,Person:Choreographer,Person:Composer,...,End date,Note:description,Subject:topic,Subject:personal name,Subject:geographic,Related resource:related,Access granted,Copyright status,Copyright holder,CC license
0,1,Object,NaN,NaN,moving image | still image,eng - English,St. Francis de los Barrios,NaN,NaN,"Waters, Joseph",...,NaN,"An indie-class, trans-genre rock opera based o...",Opera | Transgender sex workers,Saint Francis of Assisi | Pope Francis,"Tijuana (Baja California, Mexico) | Zona Norte...",IDEAS Performance Photographs via Flickr @ htt...,NaN,NaN,NaN,NaN
1,1,Component,IdeasStFrancis_2017_12_06_HQ.mp4,video-source,NaN,NaN,Program,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Component,IDEAS_St_Francis_de_los_Barrios_01_of_88.jpg,image-source,NaN,NaN,Image 1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Component,IDEAS_St_Francis_de_los_Barrios_02_of_88.jpg,image-source,NaN,NaN,Image 2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Component,IDEAS_St_Francis_de_los_Barrios_03_of_88.jpg,image-source,NaN,NaN,Image 3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, we have our existing loop which you can see in the next cell. Basically, it will make two lists, one for `Begin date`, and one for `End date`. In day-specific dates (like "2018-03-04"), both values in both lists will be the same.  

For vague dates, like "February 2018", the `Begin date` is 2018-02-01, while the `End date` would be 2018-02-28. Now you see why the above `dateparser` settings can really help us parse dates like these.  

For blank cells, as mentioned we cannot simply check the length of a cell. We need to use a `pandas` function like `pd.isnull()`. See below how we check for that, and if it is null, we simply add blanks for those cells

In [29]:
date_begin = []
date_end = []
columns = dates_df.columns
for column in columns:
    if column.lower().startswith("date"):
        for row in dates_df[column]:
            if len(str(row)) == 4:
                date_begin.append(str(row) + "-01-01")
                date_end.append(str(row) + "-12-31")
            elif len(str(row)) == 10:
                date_begin.append(str(row))
                date_end.append(str(row))
            elif pd.isnull(row) == True:
                date_begin.append(str(row) + "")
                date_end.append(str(row) + "")
            else:
                parsed_begin = parse(str(row), settings={'PREFER_DAY_OF_MONTH': 'first'})
                parsed_end = parse(str(row), settings={'PREFER_DAY_OF_MONTH': 'last'})
                date_begin.append(str(parsed_begin))
                date_end.append(str(parsed_end))
dates_df['Begin date'] = date_begin
dates_df['End date'] = date_end       

In [31]:
begin_dates = dates_df['Begin date']
begin_dates

0      2017-12-06 00:00:00
1                      nan
2                      nan
3                      nan
4                      nan
              ...         
532                    nan
533                    nan
534                    nan
535                    nan
536                    nan
Name: Begin date, Length: 537, dtype: object

In [19]:
dates_df['End date']

0      2017-12-06 00:00:00
1      2020-08-31 00:00:00
2      2020-08-31 00:00:00
3      2020-08-31 00:00:00
4      2020-08-31 00:00:00
              ...         
532    2020-08-31 00:00:00
533    2020-08-31 00:00:00
534    2020-08-31 00:00:00
535    2020-08-31 00:00:00
536    2020-08-31 00:00:00
Name: End date, Length: 537, dtype: object